In [ ]:
!pip install Flask
!pip install flask_ngrok
!pip install pyngrok==7.1.2
!pip install gtts
!ngrok authtoken 2fIxJpLgjs653mjOWz1NVxPGrN9_39Kd7f2WXEsfdX8K2fNnP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=05f5a7934adac6eb9ce6f2041ced7cf5c9dbdc0d51ac71f8cf7abc59dabab8c3
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2fIxJpLgjs653mjOWz1NVxPGrN9_39Kd7f2WXEsfdX8K2fNnP")
port=5000
ngrok.connect(port).public_url

'https://475c-34-83-139-215.ngrok-free.app'

In [ ]:
import cv2
import face_recognition
from google.colab.patches import cv2_imshow

def process_video(input_video_path, output_video_path, flag, duration_sec=4):
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Calculate the number of frames to process for the specified duration
    frames_to_process = min(int(fps * duration_sec), total_frames)

    # Use mp4v codec for MP4 format
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Create VideoWriter object to save the output video
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    label = "fake" if flag else "real"

    frame_count = 0
    while frame_count < frames_to_process:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame from BGR to RGB for face_recognition
        rgb_frame = frame[:, :, ::-1]

        # Detect face locations
        face_locations = face_recognition.face_locations(rgb_frame)

        # Draw bounding boxes and labels
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Write the frame to the output video
        out.write(frame)

        # Display the frame (optional)
        cv2_imshow(frame)  # Use cv2_imshow in Google Colab
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()




In [ ]:
from flask import Flask, render_template

import os
from flask import Flask ,  render_template , redirect, request, jsonify
template_folder='/content/drive/MyDrive/video_display_output/template'
static_folder='/content/drive/MyDrive/video_display_output/static'
app = Flask(__name__,template_folder=template_folder,static_folder=static_folder)


@app.route('/')
def index():
    return render_template('VideoUpload.html')

@app.route('/upload', methods=['POST'])
def convertvid():
    video_f = request.files.get('inpvid')
    if video_f:
      videoName = f"/content/drive/MyDrive/video_display_output/static/videos/{video_f.filename}"
      print(video_f)
      video_f.save(videoName)
      return render_template('ViewVideo.html')
    else:
      return "No file uploaded", 400



if __name__ == "__main__":
    port = 5000  # Define a port number
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import os

from flask import Flask ,  render_template , redirect, request, jsonify
template_folder='/content/drive/MyDrive/video_display_output/template'
static_folder='/content/drive/MyDrive/video_display_output/static'
app = Flask(__name__,template_folder=template_folder,static_folder=static_folder)

@app.route('/')
def index():
    return render_template('VideoUpload.html')

@app.route('/upload', methods=['POST'])
def convertvid():

    file = request.files.get('inpvid')
    print(file)
    if file:
        # Save the uploaded file to a specific location
        video_path = '/content/drive/MyDrive/video_display_output/static/videos/'
        file.save(os.path.join(video_path, file.filename))
        # Example usage
        input_video_path = os.path.join(video_path, file.filename)
        outp="output_video.mp4"
        output_video_path = os.path.join(video_path, 'output.mp4')
        flag = False  # Set to False for "real"

        process_video(input_video_path, output_video_path, flag, duration_sec=4)
        print('success')
        # Redirect to the view page with the filename as a query parameter
        return redirect(url_for('view_video', filename='output.mp4'))

    return redirect(request.url)

@app.route('/view/<filename>')
def view_video(filename):
    # video_path = f'/static/videos/{filename}'
    video_path= f"../static/videos/{filename}"
    return render_template('ViewVideo.html', video_path=video_path)

@app.route('/view')
def view2():
    return render_template('ViewVideo.html')

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:31:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:31:27] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:31:32] "GET /view HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:31:32] "GET /{url_for('static',%20filename='videos/output.mp4')} HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:32:40] "GET /static/videos/output.mp4 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:32:40] "GET /static/videos/output.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:32:41] "GET /static/videos/output.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15:33:08] "GET /static/videos/00037.mp4 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/May/2024 15